In [ ]:
import paho.mqtt.subscribe as subscribe
import pandas as pd
import os

topics = ['alextemp', 'alexhumidity', 'alextime', 'alexgas']

temperature = []
humidity = []
time = []
gas = []

n_prints = 5
csv_file_path = 'output5.csv'

file_exists = os.path.isfile(csv_file_path)
last_time = None

for n in range(0, n_prints):
    data = []
    m = subscribe.simple(topics, hostname="pf-eveoxy0ua6xhtbdyohag.cedalo.cloud", retained=False, msg_count=len(topics))
    
    for a in m:
        data.append(a.payload)
    
    temp_value = float(data[0].decode('utf-8'))
    humidity_value = float(data[1].decode('utf-8'))
    time_value = float(data[2].decode('utf-8'))
    gas_value = float(data[3].decode('utf-8'))

    if time_value != last_time:
        temperature.append(temp_value)
        humidity.append(humidity_value)
        time.append(time_value)
        gas.append(gas_value)
        last_time = time_value

        df = pd.DataFrame({
            'temperature()': [temp_value],
            'humidity': [humidity_value],
            'time(s)': [time_value],
            'gas': [gas_value]
        })

        df.to_csv('output5.csv', mode='a', header=not file_exists, index=False)
        file_exists = True
    else:
        print(f"Skipping duplicate time value: {time_value}")

print(temperature)
print(humidity)
print(time)
print(gas)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
file_path = 'output4.csv'
df = pd.read_csv(file_path)

# View the first few rows of the DataFrame
print(df.head())
# Line plot showing temperature and humidity over time
plt.figure(figsize=(10, 6))

sns.lineplot(x='time(s)', y='temperature()', data=df, label='Temperature', color='red')
sns.lineplot(x='time(s)', y='humidity', data=df, label='Humidity', color='blue') # 2 different y axis

plt.title('Temperature and Humidity over Time')
plt.xlabel('Time(s)')
plt.ylabel('Value')
plt.grid()
plt.legend()

plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the data from the CSV file
file_path = 'output4.csv'
df = pd.read_csv(file_path)

# View the first few rows of the DataFrame
print(df.head())

# Set up the figure and axes
fig, axes = plt.subplots(3, 1, figsize=(10, 18))

# Plot 1: Temperature over Time
sns.lineplot(x='time(s)', y='temperature()', data=df, ax=axes[0], label='Temperature', color='red')
axes[0].set_title('Temperature over Time')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Temperature (°C)')
axes[0].grid()
axes[0].legend()

# Plot 2: Humidity over Time
sns.lineplot(x='time(s)', y='humidity', data=df, ax=axes[1], label='Humidity', color='blue')
axes[1].set_title('Humidity over Time')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Humidity (%)')
axes[1].grid()
axes[1].legend()

# Plot 3: Gas over Time
sns.lineplot(x='time(s)', y='gas', data=df, ax=axes[2], label='Gas', color='green')
axes[2].set_title('Gas over Time')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Gas (units)')
axes[2].grid()
axes[2].legend()

# Adjust the layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

# Read the data from the CSV file
file_path = 'output4.csv'
df = pd.read_csv(file_path)

# View the first few rows of the DataFrame
print(df.head())

# Assume the time is in seconds, and we want to start at 20:00 (8:00 PM).
start_time = pd.to_datetime('2024-12-04 20:00')

# Convert time(s) from seconds to timedelta and add to the start time
df['timestamp'] = start_time + pd.to_timedelta(df['time(s)'], unit='s')

# Set up the figure and axes
fig, axes = plt.subplots(3, 1, figsize=(10, 18))

# Plot 1: Temperature over Time
sns.lineplot(x='timestamp', y='temperature()', data=df, ax=axes[0], label='Temperature', color='red')
axes[0].set_title('Temperature over Time')
axes[0].set_xlabel('Date/Time')
axes[0].set_ylabel('Temperature (°C)')
axes[0].grid()
axes[0].legend()

# Plot 2: Humidity over Time
sns.lineplot(x='timestamp', y='humidity', data=df, ax=axes[1], label='Humidity', color='blue')
axes[1].set_title('Humidity over Time')
axes[1].set_xlabel('Date/Time')
axes[1].set_ylabel('Humidity (%)')
axes[1].grid()
axes[1].legend()

# Plot 3: Gas over Time
sns.lineplot(x='timestamp', y='gas', data=df, ax=axes[2], label='Gas', color='green')
axes[2].set_title('Gas over Time')
axes[2].set_xlabel('Date/Time')
axes[2].set_ylabel('Gas (units)')
axes[2].grid()
axes[2].legend()

# Adjust the layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()


In [2]:
import pandas as pd
import sqlite3

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('output4.csv')  # Adjust the path to your CSV file

# Add new columns for experiment_id, location, and date
df['experiment_id'] = range(1, len(df) + 1)  # Start experiment_id from 1
df['location'] = "Joe's Office"  # Add location as Joe's Office
df['date'] = '2024-12-05'  # Set the date for all entries

# Display the first few rows to ensure the new columns are added
print(df.head())

# Connect to an SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('sensor_data.db')

# Create a table based on the updated DataFrame columns
conn.execute('''
    CREATE TABLE IF NOT EXISTS sensor_data (
        experiment_id INTEGER,
        temperature REAL,
        humidity REAL,
        time_s REAL,
        time INTEGER,
        location TEXT,
        date TEXT
    )
''')

# Commit the transaction to create the table
conn.commit()

# Insert the DataFrame data into the SQLite table
df.to_sql('sensor_data', conn, if_exists='replace', index=False)

# Commit the changes
conn.commit()

# Run a simple query to retrieve all data
query = 'SELECT * FROM sensor_data LIMIT 100;'
result = conn.execute(query).fetchall()

# Print the results
for row in result:
    print(row)

# Close the connection
conn.close()


   temperature()  humidity  time(s)      gas  experiment_id      location  \
0          17.86      50.0   9961.0  29881.0              1  Joe's Office   
1          17.87      50.0   9966.0  29881.0              2  Joe's Office   
2          17.86      50.0   9972.0  30009.0              3  Joe's Office   
3          17.87      50.0   9977.0  29860.0              4  Joe's Office   
4          17.87      50.0   9982.0  30073.0              5  Joe's Office   

         date  
0  2024-12-05  
1  2024-12-05  
2  2024-12-05  
3  2024-12-05  
4  2024-12-05  
(17.86, 50.0, 9961.0, 29881.0, 1, "Joe's Office", '2024-12-05')
(17.87, 50.0, 9966.0, 29881.0, 2, "Joe's Office", '2024-12-05')
(17.86, 50.0, 9972.0, 30009.0, 3, "Joe's Office", '2024-12-05')
(17.87, 50.0, 9977.0, 29860.0, 4, "Joe's Office", '2024-12-05')
(17.87, 50.0, 9982.0, 30073.0, 5, "Joe's Office", '2024-12-05')
(17.86, 50.0, 9987.0, 30138.0, 6, "Joe's Office", '2024-12-05')
(17.87, 50.0, 9992.0, 29924.0, 7, "Joe's Office", '2024-1